In [1]:
import os
import requests
import json
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
AIDEVS_API_KEY = os.environ["AIDEVS_API_KEY"]

In [3]:
client = OpenAI()

In [4]:
system_context_hint = """
You are a specialist in answering questions about a person based on a set of hints. Only exception is that when asked,
you don't return the person's name but return YES if you are certain who that is and NO if you are not. You don't take
guesses and return YES only if you are certain.

### Example
Hint 1: American politician who was born in 1961 Honolulu, Hawaii, U.S.
Asistant: NO

Hint 1: American politician who was born in 1961 Honolulu, Hawaii, U.S.
Hint 2: He or she was elected President of the United States of America in 2009.
Asistant: YES
"""

system_context_aswer = """
You are a specialist in answering questions about a person based on a set of hints. You return only the person's name
based on your best guess and the hints you are provided.

### Example
Hint 1: American politician who was born in 1961 Honolulu, Hawaii, U.S.
Hint 2: He or she was elected President of the United States of America in 2009.
Asistant: Barack Obama
"""

In [5]:
hints = []
counter = 0

while True:
    response = requests.post("https://tasks.aidevs.pl/token/whoami", json=dict(apikey=AIDEVS_API_KEY))
    response_json = json.loads(response.text)
    token = response_json["token"]

    response = requests.get(f"https://tasks.aidevs.pl/task/{token}")
    response_json = json.loads(response.text)

    hint = response_json["hint"]
    hints.append(hint)
    print(hints[counter])
    
    completion = client.chat.completions.create(
      model="gpt-4",
      messages=[
        {"role": "system", "content": system_context_hint},
        {"role": "user", "content": f"\n".join(f"Hint {idx+1}: {el}" for idx, el in enumerate(hints))}
      ]
    )

    answer = str(completion.choices[0].message.content)
    print(answer)
    if counter > 7 or answer == "YES":
        break
    counter += 1

był wegetarianinem i eksperymentował z różnymi ekstremalnymi dietami
NO
uczęszczał do college, ale rzucił szkołę po jednym semestrze
NO
Był pod wpływem Zen buddyzmu i odwiedził Indie w poszukiwaniu duchowego oświecenia
YES


In [6]:
completion = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {"role": "system", "content": system_context_aswer},
    {"role": "user", "content": f"\n".join(f"Hint {idx+1}: {el}" for idx, el in enumerate(hints))}
  ]
)

answer = str(completion.choices[0].message.content)
print(answer)

Steve Jobs


In [7]:
response = requests.post(f"https://tasks.aidevs.pl/answer/{token}", json=dict(answer=answer))
print(response.text)

{
    "code": 0,
    "msg": "OK",
    "note": "CORRECT"
}
